In [1]:
import os

In [2]:
%pwd

'c:\\Text-Summarizer-Project\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Text-Summarizer-Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml,create_directories
from pathlib import Path
from textSummarizer.logging import logger

[2024-06-19 10:24:32,178: INFO: __init__: Using CONFIG_FILE_PATH: config\config.yaml]
[2024-06-19 10:24:32,179: INFO: __init__: Using PARAMS_FILE_PATH: params.yaml]


In [7]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        logger.info(f"Initializing ConfigurationManager with config file: {config_filepath} and params file: {params_filepath}")
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [8]:
import os
import urllib.request as request
import zipfile
from textSummarizer.logging import logger
from textSummarizer.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")  

        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-06-19 10:24:35,268: INFO: 3533509639: Initializing ConfigurationManager with config file: config\config.yaml and params file: params.yaml]
[2024-06-19 10:24:35,270: INFO: common: Attempting to read YAML file: config\config.yaml]
[2024-06-19 10:24:35,272: INFO: common: YAML file: config\config.yaml loaded successfully with content: {'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://github.com/entbappy/Branching-tutorial/raw/master/summarizer-data.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}}]
[2024-06-19 10:24:35,273: INFO: common: Attempting to read YAML file: params.yaml]
[2024-06-19 10:24:35,276: INFO: common: YAML file: params.yaml loaded successfully with content: {'param1': 'value1', 'param2': 'value2'}]
[2024-06-19 10:24:35,277: INFO: common: created directory at: artifacts]
[2024-06-19 10:24:35,278: INFO: common: created directory at: artifacts/data_inge

TypeError: DataIngestionConfig.__init__() got an unexpected keyword argument 'source_URL'